<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/Amazon_Domain_Training_to_Classification_Test_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary

This example takes a roberta-base model that I did a quick domain training on (BigTMiami/test_model) and then tries to use it for doing text classification.

## To Do for Real Run

*   Set Epochs to 3 from 1
*   Set Train \ Test datasets from Test \ Dev
*   Grant Secret Access to Notebook
*   Need to use fully domain pretrained model

# Setup

In [1]:
!pip install datasets
!pip install huggingface_hub
!pip install scikit-learn
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 97.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)
from sklearn.metrics import accuracy_score,  f1_score

from datasets import load_dataset

Create function for metrics for evaluation

In [3]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

Load up the dataset for amazon helpfulness classification

In [4]:
from datasets import load_dataset
dataset = load_dataset("BigTMiami/amazon_helpfulness")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Load the tokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
#Label Settings
id2label = {0: "unhelpful", 1: "helpful"}
label2id = {"unhelpful": 0, "helpful": 1}

In [7]:
# Set Classifier Settings
classification_config = AutoConfig.from_pretrained("BigTMiami/test_model") # NEED TO CHANGE FOR REAL RUN
classification_config.classifier_dropout = 0.1 # From Paper
classification_config.num_of_labels = 2
classification_config.id2label=id2label
classification_config.label2id=label2id,
classification_config

config.json:   0%|          | 0.00/672 [00:00<?, ?B/s]

RobertaConfig {
  "_name_or_path": "BigTMiami/test_model",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "unhelpful",
    "1": "helpful"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": [
    {
      "helpful": 1,
      "unhelpful": 0
    }
  ],
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_of_labels": 2,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.38.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

Load the pretrained model - BigTMiami/test_model. This model had some pre-training done on the Amazon Domain.

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("BigTMiami/test_model",
        config=classification_config) # NEED TO CHANGE FOR REAL RUN

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BigTMiami/test_model and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
training_args = TrainingArguments(
    output_dir="amazon_domain_to_classification_test",
    learning_rate=2e-5, # Paper: this is for Classification, not domain training
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1, # NEED TO 3 FOR REAL TEST
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["test"],# NEED TO SET TO train FOR REAL TEST
    eval_dataset=dataset["dev"],# NEED TO SET TO test FOR REAL TEST
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.args

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.98,
adam_epsilon=1e-06,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_l

# Train

I do an evaluation here to see if the pre-training score is different for the domain model vs the roberta-base.

In [11]:
trainer.evaluate()

{'eval_loss': 0.6072725653648376,
 'eval_accuracy': 0.8534,
 'eval_f1_macro': 0.46045106291140603,
 'eval_runtime': 92.0747,
 'eval_samples_per_second': 54.304,
 'eval_steps_per_second': 3.399}

In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.387900,0.381573,0.853800,0.500545


TrainOutput(global_step=1563, training_loss=0.4107301593665808, metrics={'train_runtime': 761.4248, 'train_samples_per_second': 32.833, 'train_steps_per_second': 2.053, 'total_flos': 6139092143620320.0, 'train_loss': 0.4107301593665808, 'epoch': 1.0})

In [13]:
trainer.evaluate()

{'eval_loss': 0.3815734386444092,
 'eval_accuracy': 0.8538,
 'eval_f1_macro': 0.500544891175496,
 'eval_runtime': 91.7998,
 'eval_samples_per_second': 54.466,
 'eval_steps_per_second': 3.41,
 'epoch': 1.0}

In [14]:
trainer.push_to_hub()

events.out.tfevents.1712345463.95520175a3b3.974.1:   0%|          | 0.00/463 [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1712344579.95520175a3b3.974.0:   0%|          | 0.00/6.54k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BigTMiami/amazon_domain_to_classification_test/commit/f0ef79ad590437bacbdf4e70961dca382645e332', commit_message='End of training', commit_description='', oid='f0ef79ad590437bacbdf4e70961dca382645e332', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()